In [ ]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [6]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

In [7]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

In [8]:
vector = FAISS.from_documents(documents,embeddings)
retriever = vector.as_retriever()

In [9]:
retriever.invoke("how to upload a dataset")[0]

Document(id='2db8ffa7-567c-4c3c-b1fa-34e8aaca26ce', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': '\uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'language': 'en'}, page_content='\uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentLangSmith docs have moved! Find the LangSmith docs at the new LangChain Docs site.API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityLangChain ForumTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright ¬© 2025 LangChain, Inc.')

In [10]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [18]:
import os
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
# Load environment variables from .env file
load_dotenv()

# Now Python can read the key
tavily_api_key = os.getenv("TAVILY_API_KEY")

# Initialize Tavily
tavily_tool = TavilySearchResults(tavily_api_key=tavily_api_key)
tavily_tool.invoke("what is the weather in SF")

[{'title': 'Weather San Francisco in September 2025: Temperature & Climate',
  'url': 'https://en.climate-data.org/north-america/united-states-of-america/california/san-francisco-385/t/september-9/',
  'content': '| 12. September | 17 °C | 62 °F | 23 °C | 73 °F | 13 °C | 56 °F | 14 °C | 57 °F | 0.1 mm | 0.0 inch. |\n| 13. September | 17 °C | 62 °F | 23 °C | 73 °F | 13 °C | 56 °F | 14 °C | 57 °F | 0.1 mm | 0.0 inch. |\n| 14. September | 17 °C | 62 °F | 22 °C | 71 °F | 13 °C | 56 °F | 14 °C | 58 °F | 0.3 mm | 0.0 inch. |\n| 15. September | 16 °C | 61 °F | 21 °C | 70 °F | 13 °C | 55 °F | 14 °C | 58 °F | 0.0 mm | 0.0 inch. | [...] | Max. Temperature °C (°F) | 14 °C (57.3) °F | 14.9 °C (58.7) °F | 16.2 °C (61.2) °F | 17.4 °C (63.3) °F | 19.2 °C (66.5) °F | 21.5 °C (70.8) °F | 21.8 °C (71.2) °F | 22.2 °C (71.9) °F | 23.1 °C (73.6) °F | 21.3 °C (70.3) °F | 17.1 °C (62.8) °F | 13.9 °C (57.1) °F |\n| Precipitation / Rainfall mm (in) | 113 (4) | 118 (4) | 83 (3) | 40 (1) | 21 (0) | 6 (0) | 2 (0)

In [26]:
tools = [tavily_tool, retriever_tool]
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "hi! what is a langsmith?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'what is a langsmith'}`


ü¶úÔ∏èüõ†Ô∏è LangSmith








Skip to main contentLangSmith docs have moved! Find the LangSmith docs at the new LangChain Docs site.API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityLangChain ForumTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright ¬© 2025 LangChain, Inc.LangSmith is a powerful open-source framework designed to simplify the development of applications powered by large language models (LLMs). 




> Finished chain.


{'input': 'hi! what is a langsmith?',
 'output': 'LangSmith is a powerful open-source framework designed to simplify the development of applications powered by large language models (LLMs). \n\n\n'}

In [28]:
agent_executor.invoke({"input": "whats the weather in Pune?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in Pune'}`


[{'title': 'Weather Pune in September 2025: Temperature & Climate', 'url': 'https://en.climate-data.org/asia/india/maharashtra/pune-31/t/september-9/', 'content': '| Max. Temperature °C (°F) | 28.7 °C (83.7) °F | 31.3 °C (88.4) °F | 34.5 °C (94) °F | 37 °C (98.7) °F | 35.5 °C (95.9) °F | 29 °C (84.2) °F | 25.6 °C (78.1) °F | 25.4 °C (77.7) °F | 26.9 °C (80.5) °F | 29.2 °C (84.6) °F | 29.3 °C (84.8) °F | 28.4 °C (83.2) °F |\n| Precipitation / Rainfall mm (in) | 1 (0) | 2 (0) | 3 (0) | 10 (0) | 26 (1) | 247 (9) | 340 (13) | 270 (10) | 189 (7) | 81 (3) | 26 (1) | 5 (0) |\n| Humidity(%) | 46% | 41% | 35% | 40% | 55% | 79% | 88% | 88% | 85% | 67% | 53% | 48% | [...] | Min. Temperature °C (°F) | 15.1 °C (59.3) °F | 16.8 °C (62.3) °F | 19.1 °C (66.3) °F | 21 °C (69.8) °F | 22 °C (71.7) °F | 22.3 °C (72.2) °F | 21.6 °C (70.9) °F | 21.2 °C (70.1) °F | 20.8 °C (69.4) °F | 20 °C (

{'input': 'whats the weather in Pune?', 'output': '\n\n\n\n\n\n\n\n\n'}

ReAct Agent (Reasoning + Acting):
Definition:

The ReAct framework combines reasoning and acting in a single loop to handle tasks. The agent uses natural language reasoning (thinking through steps) and task actions (performing tasks like calculations or data retrieval).

The agent uses a combination of reasoning steps to guide actions in real-time, using feedback from those actions to further inform the next step in reasoning.

How it works:

Step 1: The agent receives a question or task.
Step 2: It reasons aloud (in natural language) about how to solve it.
Step 3: Based on its reasoning, it takes actions (e.g., searching a database, calculating something).
Step 4: The results of these actions are integrated into its reasoning and may trigger further actions.
Step 5: It repeats the process until it arrives at a solution.
Key points:

Combines thinking and doing (reasoning and actions).
Performs iterative steps, updating its process based on action results.
Typically handles complex decision-making scenarios.
Example:

Original task: “Calculate the total number of apples in a basket if there are 4 baskets and 7 apples in each.”
Reasoning: "I need to multiply 4 by 7 to get the total number of apples."
Action: Perform the multiplication.
Result: "There are 28 apples."
Reasoning: "I am done."

In [29]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

In [31]:
template = '''Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}'''

In [32]:
prompt = PromptTemplate.from_template(template)
search_agent = create_react_agent(llm,tools,prompt)

In [33]:
agent_executor = AgentExecutor(
    agent=search_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
)
agent_executor.invoke({"input": "Where is the hometown of the 2007 US PGA championship winner and his score?"})



> Entering new AgentExecutor chain...
Thought: I need to find information about the 2007 US PGA Championship winner.
Action: tavily_search_results_json
Action Input: 2007 US PGA Championship winner[{'title': 'Tiger Woods Lookback: 2007 PGA Championship at Southern Hills', 'url': 'https://www.pgachampionship.com/news-media/tiger-woods-lookback-2007-pga-championship-at-southern-hills', 'content': 'Corebridge.svg\nChase Logo SVG\nT-Mobile-For-Business_Logo_\n\n# Tiger Woods Lookback: 2007 PGA Championship at Southern Hills\n\nWhile four-time PGA Championship winner Tiger Woods has confirmed his heroic return to professional golf just 14 months after a horrible car accident, this year also marks the 15 year anniversary of Woods’ victory at Southern Hills Country Club – the host course for this year’s Major Championship. [...] Woods captured his fourth PGA Championship in 2007, topping Woody Austin by two shots.\n\nAustin made it interesting on the back nine making birdies on No’s 11-13, 

{'input': 'Where is the hometown of the 2007 US PGA championship winner and his score?',
 'output': 'I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access to: I now have access t

### ReAct Agent with Custom tools


In [34]:
from langchain.tools import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
# Custom tool for the Agent
@tool
def get_employee_id(name):
  """
  To get employee id, it takes employee name as arguments
  name(str): Name of the employee
  """
  fake_employees = {
    "Alice": "E001",
    "Bob": "E002",
    "Charlie": "E003",
    "Diana": "E004",
    "Evan": "E005",
    "Fiona": "E006",
    "George": "E007",
    "Hannah": "E008",
    "Ian": "E009",
    "Jasmine": "E010"}

  return fake_employees.get(name,"Employee not found")
# Custom tool for the Agent
@tool
def get_employee_salary(employee_id):
  """
  To get the salary of an employee, it takes employee_id as input and return salary
  """
  employee_salaries = {
    "E001": 56000,
    "E002": 47000,
    "E003": 52000,
    "E004": 61000,
    "E005": 45000,
    "E006": 58000,
    "E007": 49000,
    "E008": 53000,
    "E009": 50000,
    "E010": 55000
    }
  return employee_salaries.get(employee_id,"Employee not found")
# Saved React Prompt in langchain hub, we could manually type the prompt as well.
prompt = hub.pull("hwchase17/react")
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [35]:
tools = [get_employee_salary, get_employee_id]
agent = create_react_agent(llm,tools,prompt)
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor.invoke({"input":"What is the Salary of Evan?"})



> Entering new AgentExecutor chain...
Thought: I need to find Evan's employee ID first, then use that to get his salary.
Action: get_employee_id
Action Input: EvanE005Thought: Now that I have Evan's employee ID, I can get his salary.
Action: get_employee_salary
Action Input: E00545000Thought: I now know the final answer
Final Answer: 45000 




> Finished chain.


{'input': 'What is the Salary of Evan?', 'output': '45000'}